In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 라이브러리
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

# TF-IDF Vectorization 라이브러리
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# 경로 설정
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/')

In [ ]:
# 파일 불러오기
train = pd.read_csv('open/train.csv', encoding = 'utf-8')
test = pd.read_csv('open/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('open/sample_submission.csv', encoding = 'utf-8')

In [ ]:
# 텍스트 전처리 함수 정의
# 한글, 알파벳, 숫자, 그리고 문장부호 중 ;와 :을 제외하고는 공백 처리하는 함수
def alpha_num(string):
    # 정규표현식을 이용한 전처리
    string = re.sub(r"[^가-힣A-Za-z0-9;:!?]", " ", string) # 한글, 알파벳, 숫자, ;: 제외하고는 모두 공백 처리
    string = re.sub(r"\'s", " 's", string)                # 's 앞에 공백 추가
    string = re.sub(r"\'ve", " 've", string)              # 've 앞에 공백 추가
    string = re.sub(r"\'t", " 't", string)                # 't 앞에 공백 추가
    string = re.sub(r"\'re", " 're", string)              # 're 앞에 공백 추가
    string = re.sub(r"\'d", " 'd", string)                # 'd 앞에 공백 추가
    string = re.sub(r"\'ll", " 'll", string)              # 'll 앞에 공백 추가
    string = re.sub(r",", " ", string)                    # 쉼표 제거
    string = re.sub(r"!", " ! ", string)                  # 느낌표 ! 앞뒤로 공백 추가
    string = re.sub(r"\(", " ( ", string)                 # 여는 괄호 ( 앞뒤로 공백 추가
    string = re.sub(r"\)", " ) ", string)                 # 닫는 괄호 ) 앞뒤로 공백 추가
    string = re.sub(r"\?", " ? ", string)                 # 물음표 ? 앞뒤로 공백 추가
    string = re.sub(r"\s{2,}", " ", string)               # 2개 이상의 공백을 공백 1개로 변경
    string = re.sub(r"\'{2,}", " ", string)               # 2개 이상의 작은 따옴표 제거
    string = re.sub(r"\'", " ", string)                   # 작은 따옴표 ' 제거
    string = re.sub(r";", " ; ", string)                  # 세미콜론 ; 앞뒤로 공백 추가
    string = re.sub(r"\"", "", string)                    # 큰 따옴표 " 제거
    string = re.sub(r"\.", "  ", string)                  # 온점 제거

    return string


# Train 데이터 전처리
train['text']=train['text'].apply(alpha_num)

In [ ]:
# 불용어 제거 함수 정의
def remove_stopwords(text):
    # 최종 텍스트들을 저장할 배열 생성
    final_text = []
    # # text을 split()하여 i로 전달
    for i in text.split():
        # 데이콘 제공 불용어 데이터와 겹치지 않는 데이터의 공백을 없애고 소문자로 변환
        if i.strip().lower() not in stopwords:
            # final_text 배열에 공백 제거 후 저장
            final_text.append(i.strip())

    # " "문자열의 문자와 문자 사이에 삽입하는 join 메소드
    return " ".join(final_text)


# DACON 제공 기준 불용어 목록 -> 최종적으로 사용하지 않음
stopwords = [ "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves"]

In [ ]:
# 텍스트 데이터 전처리

# 소문자로 변경
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()

# 불용어 함수 적용
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

## 텍스트 데이터 시각화를 위한 전처리 과정

In [ ]:
# 작가별 text 나누는 함수 정의
author_0 = train.loc[train['author'] == 0, 'text']
author_1 = train.loc[train['author'] == 1, 'text']
author_2 = train.loc[train['author'] == 2, 'text']
author_3 = train.loc[train['author'] == 3, 'text']
author_4 = train.loc[train['author'] == 4, 'text']

In [ ]:
# 시리즈를 문자열 데이터로 바꾸는 함수 정의
def series_list_to_str(a):
  # 시리즈를 리스트로 변환
  a.to_list()
  # 리스트를 문자열로 변환
  return " ".join([str(_) for _ in a])

# 작가 별 단어 사용 시각화를 위해 따로 저장
str_author_0 = series_list_to_str(author_0)
str_author_1 = series_list_to_str(author_1)
str_author_2 = series_list_to_str(author_2)
str_author_3 = series_list_to_str(author_3)
str_author_4 = series_list_to_str(author_4)

In [ ]:
tokenizer = Tokenizer()

# 단어 빈도 수에 따라 랭킹을 부여하는 함수 정의
def fitontexts(words):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts([words])
  return tokenizer.word_index

# 각 작가 별로 빈도 수가 높은 단어 순으로 저장
author0_index = fitontexts(author_word_0)
author1_index = fitontexts(author_word_1)
author2_index = fitontexts(author_word_2)
author3_index = fitontexts(author_word_3)
author4_index = fitontexts(author_word_4)

In [ ]:
# 시각화를 위한 라이브러리 
import seaborn as sns
from collections import  Counter
# figure 사이즈 지정
rcParams['figure.figsize'] = (12, 8)

# 각 작가 별 상위 30개 단어 시각화
def plot_top_non_stopwords_barchart(text):    
    new = text.str.split()
    new = new.values.tolist()
    corpus = [word for i in new for word in i]
    counter = Counter(corpus)
    most = counter.most_common()
    x, y=[], []
    # 상위 30개 단어 시각화
    for word,count in most[:30]:
          x.append(word)
          y.append(count)
            
    sns.barplot(x=y, y=x)

# Author 4의 탑 30개 단어 시각화 - 같은 방식으로 나머지 작가들도 시각화  
plot_top_non_stopwords_barchart(author_4)


In [ ]:
# 모든 작가의 상위 10개 단어 시각화
fig = plt.figure(figsize=(30,20))

def plot_top_non_stopwords_barchart2(text):
    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]
    counter=Counter(corpus)
    most=counter.most_common()
    x, y=[], []
    # 상위 10개 단어 시각화
    for word,count in most[:10]:
          x.append(count)
          y.append(word)
    return x, y

for i in range(5):
    x,y = plot_top_non_stopwords_barchart2(train[train['author']==i]['text'])
    ax = fig.add_subplot(3, 2, i+1)
    ax.set_title(i)
    sns.barplot(x=y,y=x)

## 시각화 결과
* 실험 1: 불용어를 포함하여 작가 별 단어 사용 시각화
* 실험 2: 불용어를 제외한 작가 별 단어 사용 시각화
* 실험 3: 불용어를 제외한 후 특수 문장부호 (;, :, ?, !) 포함 시각화
* 실험 4(최종): 불용어를 제외한 후 콜론과 세미콜론만 포함 시각화 

## 소설 작가 분류를 위한 코드 작성
* 불용어의 경우 기존 DACON 코드보다 NLTK 라이브러리가 더 자세하고 효과적이라는 결론으로 NLTK 불용어 샘플을 활용하여 모델 작성


In [ ]:
# 파일 다시 불러오기
train = pd.read_csv('open/train.csv', encoding = 'utf-8')
test = pd.read_csv('open/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('open/sample_submission.csv', encoding = 'utf-8')

In [ ]:
# NLTK 라이브러리에서 제공하는 불용어 리스트 사용
from nltk.corpus import stopwords
nltk.download('stopwords')

# nltk 라이브러리에서 제공하는 영어 불용어 샘플을 변수에 저장
stw = set(stopwords.words('english'))

# 불용어 제거 함수 정의
def remove_stopwords(text):
    # 최종 텍스트들을 저장할 배열 생성
    final_text = []
    # text을 split()하여 i로 전달
    for i in text.split():
        # 불용어 데이터와 겹치지 않는 데이터의 공백을 없애고 소문자로 변환
        if i.strip().lower() not in stw:
            # final_text 배열에 공백 제거 후 저장
            final_text.append(i.strip())
    
    # " "문자열의 문자와 문자 사이에 삽입하는 join 메소드
    return " ".join(final_text)
  

In [ ]:
# 전처리 함수의 경우 기존의 alpha_num을 그대로 사용

#소문자로 변경
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()

# 기존의 alpha_num과 불용어 함수 적용
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [ ]:
# 토크나이징을 위해 필요한 punkt 설치
nltk.download('punkt')

# 워드 토큰화 + 스팀, 레만 
# punk_sent_tokenize = PunktSentenceTokenizer()
# sent_tokenize = sent_tokenize()
def preprocess_text(text):
        corpus=[]
        stem=PorterStemmer()
        lem=WordNetLemmatizer()
        for news in text:
            # 필요할 시 word_tokenize가 아닌 다른 라이브러리 사용 가능
            words=[w for w in word_tokenize(news) 
                   # 불용어 샘플 활용시 주석 해제 후 적용
                   # stopw = set(stopwords.words('english'))
                   # if (w not in stopw)
                  ]
            # stemming: PorterStemmer 사용 - stem
            # lemmatization: WordNetLemaatizer 사용 - lem
            # words=[lem.lemmatize(w) for w in words if len(w)>2]
            corpus.append(words)
        return corpus

In [ ]:
# corpus에 최종 전처리된 데이터를 넣음 + 예시 출력
corpus = preprocess_text(train['text'])
# corpus[:5]

In [ ]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [ ]:
# TF-IDF를 통한 Vectorization
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(X_train)

# TF-IDF를 통한 Vectorization 결과
# print(vectorizer.get_feature_names())
# print(vectors.toarray())

In [ ]:
# 파라미터 값 설정
vocab_size = 53968
embedding_dim = 16
max_length = 212
padding_type='post'
#oov_tok = "<OOV>"

In [ ]:
# tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
# 데이터 sequence 변환 및 패딩 적용
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

## 임베딩을 위한 코드
### Word2Vec Embedding

In [ ]:
# Word2Vec 임베딩을 하기 위한 코드

import gensim

# word2vec이 아직 없다면 주석을 해제하고 설치
# !wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz" 

# word2vec 모델 로드
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
# Word2Vec의 경우 300 차원의 임베딩 매트릭스 생성
embedding_matrix = np.zeros((vocab_size, 300))

# 토큰 별로 벡터를 구하는 함수
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

# 토큰들의 벡터를 임베딩 메트릭스에 저장
for word, i in word_index.items(): 
    temp = get_vector(word) 
    if temp is not None: 
        embedding_matrix[i] = temp

### Glove Embedding 

In [ ]:
# 임베딩 글로브
vocab = nltk.FreqDist(np.hstack(train['text']))

# Glove가 아직 없다면 주석을 해제하고 설치
# !wget "https://drive.google.com/file/d/1yHGtccC2FV3_d6C6_Q4cozYSOgA7bG-e/view" 

glove = dict()

# Word Embeddings 폴더 안의 glove.txt 
f = open('word-embeddings/glove/glove.txt')
for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    glove[word] = vector

f.close()

In [ ]:
# Glove의 경우 100차원의 임베딩 매트릭스 생성
embedding_matrix = np.zeros((len(vocab), 100)) 

for index, word in enumerate(vocab.items()):
    # 토큰이 이미 사전학습된 glove의 목록에 존재하는 경우
    if word[0] in glove:
        # 해당 토큰의 벡터를 이용해 임베딩 메트릭스에 저장
        embedding_vector = glove[word[0]]
        embedding_matrix[index] = embedding_vector

## 자연어 처리 모델 구현

### CNN 모델
* CNN 모델은 Word2Vec 임베딩 기준으로 올린다. 
* Glove 임베딩 역시 동일하게 임베딩한 후 임베딩 계층에 그대로 넣으면 된다
* 이하의 다른 모델들 역시 Word2Vec 임베딩 조합으로 작성

In [ ]:
# Word2Vec 임베딩 + CNN 모델
# Glove 역시 word2vec과 동일한 방식으로 구현

# 모델링을 위한 텐서플로 케라스 라이브러리
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam, RMSProp
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 드롭아웃 값 정의
DROP_OUT = 0.5

# Tensorflow Keras NLP 모델 생성 - 파이프라인 제공 코드에서 Embedding 레이어만 변경
model = Sequential([
    # Word2Vec 기반 Embedding 레이어 - 위에서 정의한 word2vec 임베딩 메트릭스 활용
    # 마찬가지로 glove 방식 또한 사전 정의한 임베딩 메트릭스 활용
    Embedding(vocab_size, 300,weights = [embedding_matrix], input_length = max_length),
    # Dropout 레이어
    Dropout(DROP_OUT),
    # 컨볼루션 레이어
    Conv1D(filters = 128, kernel_size = 7, padding="valid", activation="relu", strides=3),   
    Conv1D(filters = 128, kernel_size = 7, padding="valid", activation="relu", strides=3),  
    Conv1D(filters = 128, kernel_size = 7, padding="valid", activation="relu", strides=3),   
    # MaxPooling 레이어
    GlobalMaxPooling1D(),
    # Dense 레이어 (relu 활성화 함수)
    Dense(128, activation='relu'),
    # Dropout 레이어
    Dropout(DROP_OUT),
    # Dense 레이어 (softmax 활성화 함수)
    Dense(5, activation='softmax')
])

# compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# model summary
model.summary()

In [ ]:
# 모델 학습
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

In [ ]:
# fit model

es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
num_epochs = 20

history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2, validation_split=0.2, callbacks=[es])

### LSTM 모델
* CNN과 동일하게 텐서플로 케라스를 활용해 LSTM 모델 구축했다.

In [ ]:
# Word2Vec 임베딩 + LSTM 레이어를 사용한 모델링
model2 = Sequential([Embedding(vocab_size, embedding_dim, input_length =max_length),
        # LSTM 레이어 3층
        tf.keras.layers.LSTM(units = 64, return_sequences = True),
        tf.keras.layers.LSTM(units = 64, return_sequences = True),
        tf.keras.layers.LSTM(units = 64),
        # Dense 레이어 (softmax 활성화 함수)
        Dense(5, activation='softmax')    # 결과값이 0~4 이므로 Dense(5)
    ])

model2.compile(loss= 'categorical_crossentropy', 
              optimizer= 'adam',
              metrics = ['accuracy']) 

model2.summary()

## Bi-LSTM 모델
* Bi-LSTM의 경우 처음에는 위의 다른 모델들과 같은 relu 활성화 함수와 adam 옵티마이저를 사용했지만 이후 다양한 활성화 함수와 옵티마이저를 사용하면서 비교해보았다.
* 여기에 올린 코드는 최종적으로 실험했던 (가장 결과값이 좋았던) swish 활성화 함수와 RMSProp 옵티마이저를 사용한 코드이다.

In [ ]:
# Word2Vec 임베딩 + 양방향LSTM 모델
# 다른 모델들과 동일하게 케라스 텐서플로 활용

model = tf.keras.Sequential([
    # 임베딩 레이어
    tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length),
    # Bi-LSTM 레이어
    tf.keras.layers.Bidirectional(LSTM(10, return_sequences=False)),
    # Dense 레이어 (swish 활성화 함수)
    tf.keras.layers.Dense(24, activation='swish'),
		# Dense 레이어 (softmax 활성화 함수) - 5중 분류이므로 출력값 유닛을 5로 맞추어 준다.
    tf.keras.layers.Dense(5, activation='softmax')
])

# compile model - RMSProp 옵티마이저 사용
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

# model summary
print(model.summary())

## BERT 모델
* BERT 모델의 경우 DACON 공유 코드를 사용하여 구축하였으나 약 9시간이 걸친 학습 결과 런타임이 끊어짐으로 인해 완료하지 못했다.
* BERT 모델부터는 사전 학습된 모델이 아닌 이상 개인 컴퓨터로 돌리기에는 현실적으로 무리가 있음을 느꼈다.
* BERT의 경우 구현 방식이 상이하여 위의 코드와 다르게 새롭게 작성하였다.
* 학습을 완료하지는 못했으나 0.9498의 매우 높은 var_accuracy를 확인했다.

In [ ]:
# bert encode 함수
def bert_encode(texts, tokenizer, max_len=max_len):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
trn = bert_encode(train.text.values, tokenizer, max_len=max_len)
tst = bert_encode(test.text.values, tokenizer, max_len=max_len)
y = train['author'].values
print(trn[0].shape, tst[0].shape, y.shape)

In [ ]:
# Bert 모델 구현
def get_model(bert_layer, max_len=max_len):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    x = Dense(256, activation='relu')(clf_output)
    out = Dense(n_class, activation='softmax')(x)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [ ]:
p_val = np.zeros((trn[0].shape[0], n_class))
p_tst = np.zeros((tst[0].shape[0], n_class))

for i, (i_trn, i_val) in enumerate(cv.split(trn[0], y), 1):
    print(f'training model for CV #{i}')
    es = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=2,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
    
    clf = get_model(bert_layer, max_len=max_len)
    if i == 1:
        print(clf.summary())
        
    clf.fit([x[i_trn] for x in trn], 
            to_categorical(y[i_trn]),
            validation_data=([x[i_val] for x in trn], to_categorical(y[i_val])),
            epochs=10,
            batch_size=8,
            callbacks=[es])
    p_val[i_val, :] = clf.predict([x[i_val] for x in trn])
    p_tst += clf.predict(tst) / n_fold
    
    del clf
    clear_session()
    gc.collect()

### BERT 모델 결과값

<img width="849" alt="bert1" src="https://user-images.githubusercontent.com/28593767/116551524-fd6a6780-a932-11eb-83a1-580df38a2e19.png">

<img width="1175" alt="bert2" src="https://user-images.githubusercontent.com/28593767/116551540-00655800-a933-11eb-9faf-831b3437c03f.png">